<a href="https://colab.research.google.com/github/AKDW26/Invoice-Extraction/blob/main/Invoice_OCR_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pytesseract pillow kaggle

In [2]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.6 MB/s eta 0:00:00


In [3]:
import pytesseract
import re
import os
import json
from PIL import Image
from pathlib import Path

In [4]:
# Setup Kaggle credentials
def setup_kaggle():
    # Create kaggle directory
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(exist_ok=True)

    # Create kaggle.json file
    credentials = {
        "username": "akshaydwivedi26",
        "key": "c82930f8e47af96095e0257c1140d5c6"
    }

    kaggle_file = kaggle_dir / 'kaggle.json'
    with open(kaggle_file, 'w') as f:
        json.dump(credentials, f)

    # Set permissions
    os.chmod(kaggle_file, 0o600)

    # Set environment variables
    os.environ['KAGGLE_USERNAME'] = "akshaydwivedi26"
    os.environ['KAGGLE_KEY'] = "97593a000178282942181065d850ef46"

    print("Kaggle setup complete!")

In [5]:
# Download dataset
def get_data():
    setup_kaggle()
    import kaggle

    try:
        kaggle.api.dataset_download_files('urbikn/sroie-datasetv2', path='./data', unzip=True)
        print("Data downloaded!")
    except Exception as e:
        print(f"Download failed: {e}")
        print("You can manually download from: https://www.kaggle.com/datasets/urbikn/sroie-datasetv2")

In [6]:
# Extract text from image
def get_text(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

In [7]:
# find invoice number
def find_invoice_number(text):
    patterns = [r'invoice.*?(\d+)', r'inv.*?(\d+)', r'#(\d+)']
    for pattern in patterns:
        match = re.search(pattern, text.lower())
        if match:
            return match.group(1)
    return "Not found"

In [8]:
# Find date
def find_date(text):
    pattern = r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    return "Not found"

In [9]:
# Find total amount
def find_total(text):
    patterns = [r'total.*?(\d+\.?\d*)', r'\$(\d+\.?\d*)']
    for pattern in patterns:
        match = re.search(pattern, text.lower())
        if match:
            return match.group(1)
    return "Not found"

In [10]:
# Main extraction function
def extract_invoice_data(image_path):
    # Get text from image
    text = get_text(image_path)

    # Extract fields
    result = {
        'invoice_number': find_invoice_number(text),
        'date': find_date(text),
        'total': find_total(text),
        'line_items': find_line_items(text),  # NEW: Line items
        'full_text': text[:200] + "..." if len(text) > 200 else text
    }

    return result

In [11]:
# Extract line items
def find_line_items(text):
    lines = text.split('\n')
    line_items = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Pattern 1: Item Qty Price Total
        match1 = re.search(r'([A-Za-z\s]+)\s+(\d+)\s+(\d+\.?\d*)\s+(\d+\.?\d*)', line)
        # Pattern 2: Item $Price x Qty = $Total
        match2 = re.search(r'([A-Za-z\s]+)\s+\$(\d+\.?\d*)\s+x\s+(\d+)\s+=\s+\$(\d+\.?\d*)', line)

        if match1:
            line_items.append({
                'item': match1.group(1).strip(),
                'qty': match1.group(2),
                'price': match1.group(3),
                'total': match1.group(4)
            })
        elif match2:
            line_items.append({
                'item': match2.group(1).strip(),
                'qty': match2.group(3),
                'price': match2.group(2),
                'total': match2.group(4)
            })

    return line_items

In [12]:
# Convert PDF to images
def pdf_to_images(pdf_path):
    try:
        import fitz  # PyMuPDF
        doc = fitz.open(pdf_path)
        images = []

        for page_num in range(len(doc)):
            page = doc[page_num]
            pix = page.get_pixmap()
            img_path = f"page_{page_num + 1}.png"
            pix.save(img_path)
            images.append(img_path)

        doc.close()
        print(f"Converted PDF to {len(images)} images")
        return images
    except:
        print("Install PyMuPDF: pip install PyMuPDF")
        return []

In [13]:
# Add custom field extraction
def add_custom_field(text, field_name, pattern):
    match = re.search(pattern, text.lower())
    if match:
        return match.group(1)
    return "Not found"

In [14]:
# Train model (simple pattern learning)
def train_model(training_data):
    """
    Training data format: [{'text': '...', 'labels': {'field': 'value'}}]
    This learns new patterns from training examples
    """
    learned_patterns = {}

    for example in training_data:
        text = example['text']
        labels = example['labels']

        for field, value in labels.items():
            if field not in learned_patterns:
                learned_patterns[field] = []

            # Find where the value appears in text and create pattern
            if value in text.lower():
                # Simple pattern learning - find words before the value
                words_before = text.lower().split(value.lower())[0].split()
                if words_before:
                    pattern = f"{words_before[-1]}.*?([a-z0-9\s]+)"
                    learned_patterns[field].append(pattern)

    return learned_patterns

In [15]:
# Use trained patterns
def extract_with_trained_model(text, learned_patterns):
    results = {}
    for field, patterns in learned_patterns.items():
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                results[field] = match.group(1).strip()
                break
        if field not in results:
            results[field] = "Not found"

    return results

In [16]:
# Example usage:
if __name__ == "__main__":
    # Download data
    get_data()

    # Test with an image
    image_path = "/content/data/SROIE2019/test/img/X51005200931.jpg"

    # Extract basic fields
    data = extract_invoice_data(image_path)
    print("Extracted Data:", json.dumps(data, indent=4))

    # Add custom field (example: company name)
    text = get_text(image_path)
    company = add_custom_field(text, "company", r'company.*?([a-z\s]+)')
    print("Company:", company)

Kaggle setup complete!
Dataset URL: https://www.kaggle.com/datasets/urbikn/sroie-datasetv2
Data downloaded!
Extracted Data: {
    "invoice_number": "Not found",
    "date": "09/02/2078",
    "total": "411.50",
    "line_items": [
        {
            "item": "GA",
            "qty": "4",
            "price": "85.00",
            "total": "6.00"
        },
        {
            "item": "Ge",
            "qty": "4",
            "price": "1900",
            "total": "4"
        }
    ],
    "full_text": " \n\nPERNIAGAAN ZHENG HU!\nJMOs25956-V\nNO.8@ JALAN PERMAS 9/6\nBANDAR BARU PERMAS JAYA\n81780 JOHOR BAHRU\nTEL: 07-386 7524 FAX : 07-388 3798\nGST NO : DoNROOBGa24\n\nSIMPLIFIED TAX INVOICE\n\n \n\n  \n\n \n\nSalesper..."
}
Company: Not found


In [17]:
import pandas as pd
from tqdm import tqdm

def process_all_images(image_dir):
    all_data = []
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    for image_file in tqdm(image_files, desc="Processing Images"):
        image_path = os.path.join(image_dir, image_file)
        data = extract_invoice_data(image_path)
        data['filename'] = image_file  # Add filename for reference
        all_data.append(data)

    return pd.DataFrame(all_data)

# Process all images in the test directory
image_dir = "/content/data/SROIE2019/test/img/"
df = process_all_images(image_dir)

# Display the extracted data
display(df)

Processing Images: 100%|██████████| 347/347 [06:47<00:00,  1.17s/it]


,invoice_number,date,total,line_items,full_text,filename
0,782967,29/08/2017,14.69,[],\n\nGARDENIA BAKERIES (KL) SDN BHD (13938...,X51006555835.jpg
1,Not found,02/03/2018,6,"[{'item': 't', 'qty': '600', 'price': '5', 'to...",3-1707067\n\nF&P PHARMACY\n(002309592-P)\nNO.2...,X51005365187.jpg
2,18291,28-14-18,57,[],"79 SPEED MART 5/8 (519537-x)\nLOT FAY, 2811, J...",X51005764161.jpg
3,Not found,19/06/17,23.80,[],Aneka Intertrade\n\n \n\n* GST Reg mo C0153\...,X51006392167.jpg
4,1201803090220,09/03/18,6,[],\n\nBENS INDEPENDENT GROCER SDN. BHD\n(913144...,X51005444040.jpg
...,...,...,...,...,...,...
342,100,12-21-22,6,[],\n\n31203 045—\n\nMR. O.1.Y. (M) SDN BHD\n(CO...,X51005757308.jpg
343,18050202160376,Not found,8.20,[],EpSp04\n\nUNIHAKKA INTERNATIONAL SDN BHD\n02 M...,X51007231276.jpg
344,7,03/06/2018,61.70,[],NASI KANDAR PELITA\nPELITA SAMUDRA PERTAMA (M}...,X51007579714.jpg
345,Not found,15/01/2018,4,"[{'item': 'SR', 'qty': '6', 'price': '10.50', ...",3 \P O41 olf *\n\n©\n\nSYARIKAT PERNIAGAAN GIN...,X51005676548.jpg
